# How to Replace Raster Cell Values with Values from A Different Raster Datset

Sometimes you have many bad pixels in a LS scene that you wish to replace or fill in with pixels from another scene. In this lesson you will learn how to replace pixels in one scene with those from another using Numpy.

To begin, open both of the pre-fire raster stacks, the cloud free and the cloud with cold spring fire data.

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
from matplotlib.colors import ListedColormap
import seaborn as sns
import numpy as np
import numpy.ma as ma
import pandas as pd
from shapely.geometry import mapping, box
import geopandas as gpd
import rasterio as rio
from rasterio.plot import plotting_extent
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em

sns.set_style("white")
sns.set(font_scale=1.5)

# Download data and get work dir
data = et.data.get_data("cold-springs-fire")
data_2 = et.data.get_data("cs-test-landsat")
os.chdir(os.path.join(et.io.HOME, "earth-analytics"))

First, make a masked raster stack of cold springs cloud data.

In [ ]:
# Stack the LS pre fire data
landsat_paths_pre_path = os.path.join("data", "cold-springs-fire", "landsat_collect", 
                                      "LC080340322016070701T1-SC20180214145604", "crop", 
                                      "*band*.tif")

landsat_paths_pre = glob(landsat_paths_pre_path)
landsat_paths_pre.sort()

landsat_pre_cloud, landsat_pre_meta = es.stack(
    landsat_paths_pre, nodata=-9999)

# Calculate bounds object
landsat_pre_cloud_ext_bds = rio.transform.array_bounds(
    landsat_pre_cloud.shape[1],
    landsat_pre_cloud.shape[2],
    landsat_pre_meta["transform"])

# Open the pixel_qa layer for your LS scene
landsat_pre_cl_path = os.path.join("data", "cold-springs-fire", "landsat_collect", 
                                   "LC080340322016070701T1-SC20180214145604", "crop", 
                                   "LC08_L1TP_034032_20160707_20170221_01_T1_pixel_qa_crop.tif")

with rio.open(landsat_pre_cl_path) as landsat_pre_cl:
    landsat_qa = landsat_pre_cl.read(1)
    
# GEnerate array of all possible cloud/shadow values
cloud_shadow = [328, 392, 840, 984, 1350]
cloud = [352, 368, 416, 432, 480, 864, 880, 928, 944, 992]
high_confidence_cloud = [480, 992]

vals_to_mask = cloud_shadow + cloud + high_confidence_cloud

# Call the earthpy mask function using pixel QA layer
landsat_pre_cloud_masked = em.mask_pixels(landsat_pre_cloud, landsat_qa,
                                         vals=vals_to_mask)

Plot the data to ensure that the cloud covered pixels are masked